In [9]:
import numpy as np
from math import sqrt

import datetime
import csv
import matplotlib.pyplot as plt

from pyspark.mllib.clustering import KMeans, KMeansModel

In [10]:
with open('data/detroit.csv') as f:
    csv_file = csv.reader(f)
    next(csv_file)
    crime_list = []
    for row in csv_file:
        crime_list.append(row)

In [11]:
data = []
for row in crime_list:
    crime_date = datetime.datetime.strptime(row[2], '%m/%d/%Y').date()
    crime_pos  = np.array([float(row[-2]), float(row[-1])])
    crime_type = row[3].split('-')[0].strip()
    crime_type = crime_type.split('(')[0].strip()
    crime_type = crime_type.split(',')[0].strip()
    data.append([crime_date, crime_pos, crime_type])

In [12]:
data_rdd = sc.parallelize(data)
histogram_year_rdd = data_rdd.map(lambda x: (x[0].month, 1)).reduceByKey(lambda x, y: x + y).sortByKey()
year_count = histogram_year_rdd.collect()
histogram_category_rdd = data_rdd.map(lambda x: (x[2], 1)).reduceByKey(lambda x, y: x + y).sortBy(lambda x: -x[1])
category_count = histogram_category_rdd.collect()

In [13]:
top_categories = [x[0] for x in histogram_category_rdd.take(11)[1:]]

In [14]:
def selection(item, choices):
    return choices is None or item in choices

In [15]:
def cluster(date_range, category_types, num_clusters):
    #Label data
    filtered_rdd = data_rdd.filter(lambda x: selection(x[0].month, date_range) and selection(x[2], category_types))
    pos_rdd = filtered_rdd.map(lambda x: x[1])
    clusters = KMeans.train(pos_rdd, num_clusters, maxIterations=100, initializationMode="random")
    label_rdd = pos_rdd.map(lambda x: [x, clusters.centers[clusters.predict(x)]])
    
    #Cluster and distribution
    return label_rdd, clusters

In [16]:
def sse(point, center):
    return sqrt(sum([i**2 for i in (point - center)]))

In [17]:
def error_plot(dates, categories, cluster_range):
    error = []
    for num_cluster in cluster_range:
        label_rdd, clusters = cluster(dates, categories, num_cluster)
        error.append(label_rdd.map(lambda x: sse(x[0], x[1])).reduce(lambda x, y: x + y))
    return error

In [18]:
l = len(top_categories)
for category in top_categories:
    print category

LARCENY
ASSAULT AND BATTERY/SIMPLE ASSAULT
DAMAGE TO PROPERTY
BURGLARY
AGG/FEL ASSAULT
VEHICLE THEFT
WARRANTS
FRAUD
ROBBERY
TELEPHONE USED FOR HARASSMENT


In [20]:
errors = error_plot(None, '[DAMAGE TO PROPERTY]', range(1,21))
plt.plot(errors)

Traceback (most recent call last):
  File "/usr/lib/python2.7/SocketServer.py", line 295, in _handle_request_noblock
ERROR:py4j.java_gateway:Error while sending or receiving.
Traceback (most recent call last):
  File "/usr/local/bin/spark-1.3.1-bin-hadoop2.6/python/lib/py4j-0.8.2.1-src.zip/py4j/java_gateway.py", line 479, in send_command
    raise Py4JError("Answer from Java side is empty")
Py4JError: Answer from Java side is empty
    self.process_request(request, client_address)
  File "/usr/lib/python2.7/SocketServer.py", line 321, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python2.7/SocketServer.py", line 334, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python2.7/SocketServer.py", line 649, in __init__
    self.handle()
  File "/usr/local/bin/spark-1.3.1-bin-hadoop2.6/python/pyspark/accumulators.py", line 231, in handle
    num_updates = read_int(self.rfile)
  File "/usr/local/bin/spark-1.3

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 45800)
----------------------------------------


Py4JNetworkError: An error occurred while trying to connect to the Java server